# Data Preparation 
______________

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from typing import Tuple

In [2]:
train = pd.read_csv('../data/raw/application_train.csv')
test = pd.read_csv('../data/raw/application_test.csv')

print("Train shapes : {}, Test shapes : {}".format(train.shape, test.shape))
train

Train shapes : (307511, 122), Test shapes : (48744, 121)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
#@Nikos Tavoularis
def missing_values_table(df:pd.DataFrame)->pd.DataFrame:
    """
    Counts msising value and percentage of missing values in columns of the dataframe in decsending order
    
    Parameters
    ----------
    df
        pandas dataframe to count missing values.
 
    Returns
    -------
    dataframe with name of columns containing missing values, number of missing values, % of missing values
        pd.DataFrame
    """
    
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    print(mis_val_table_ren_columns)
   
    return mis_val_table_ren_columns

In [4]:
def delete_missing_values_cols(df_train:pd.DataFrame,df_test:pd.DataFrame, threshold: float)-> Tuple[pd.DataFrame,pd.DataFrame]:
   """
   Deletes columns which % of missing values is higher than threshold specified by user
    
    Parameters
    ----------
    df_train
        train dataframe.
    df_test
        test dataframe.
    threshold
        threshold of % of missing value defined by user 
 
    Returns
    -------
    dataframes with corresponding columns deleted
        pd.DataFrame,pd.DataFrame
   """
    
   missing_values_summary = missing_values_table(df_train)
   deleting_col_names = missing_values_summary[missing_values_summary['% of Total Values']>= threshold].index.values
   new_train_df=df_train.drop(deleting_col_names, axis=1, inplace=False)
   new_test_df=df_test.drop(deleting_col_names, axis=1, inplace=False)
   print("We have deleted according to your threshold "+str(len(deleting_col_names))+" columns,\n here's the new Dataframe")
   return new_train_df, new_test_df
   


In [5]:
#@Will Koehrsen
def numerizer(df_train:pd.DataFrame,df_test:pd.DataFrame)->Tuple[pd.DataFrame,pd.DataFrame]:
    """
    Function which encodes object type column created by Will Koehrsen

    Parameters
    ----------
    df_train
        train dataframe.
    df_test
        mtest dataframe.
 
    Returns
    -------
    dataframes with encoded columns
        pd.DataFrame,pd.DataFrame
    """
    
    # Create a label encoder object
    le = LabelEncoder()
    le_count = 0

    # Iterate through the columns
    for col in df_train:
        if df_train[col].dtype == 'object':
            # If 2 or fewer unique categories
            if len(list(df_train[col].unique())) <= 2:
                # Train on the training data
                le.fit(df_train[col])
                # Transform both training and testing data
                df_train[col] = le.transform(df_train[col])
                df_test[col] = le.transform(df_test[col])
                
                # Keep track of how many columns were label encoded
                le_count += 1
                
    print('%d columns were label encoded.' % le_count)
    # one-hot encoding of categorical variables
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)

    print('Training Features shape: ', df_train.shape)
    print('Testing Features shape: ', df_test.shape)

    return df_train,df_test

In [6]:
missing_values_summary = missing_values_table(train)
print(missing_values_summary)

Your selected dataframe has 122 columns.
There are 67 columns that have missing values.
                          Missing Values  % of Total Values
COMMONAREA_MEDI                   214865               69.9
COMMONAREA_AVG                    214865               69.9
COMMONAREA_MODE                   214865               69.9
NONLIVINGAPARTMENTS_MEDI          213514               69.4
NONLIVINGAPARTMENTS_MODE          213514               69.4
...                                  ...                ...
EXT_SOURCE_2                         660                0.2
AMT_GOODS_PRICE                      278                0.1
AMT_ANNUITY                           12                0.0
CNT_FAM_MEMBERS                        2                0.0
DAYS_LAST_PHONE_CHANGE                 1                0.0

[67 rows x 2 columns]
                          Missing Values  % of Total Values
COMMONAREA_MEDI                   214865               69.9
COMMONAREA_AVG                    214865         

In [7]:
delete_missing_values_cols(train,test,50.0)

Your selected dataframe has 122 columns.
There are 67 columns that have missing values.
                          Missing Values  % of Total Values
COMMONAREA_MEDI                   214865               69.9
COMMONAREA_AVG                    214865               69.9
COMMONAREA_MODE                   214865               69.9
NONLIVINGAPARTMENTS_MEDI          213514               69.4
NONLIVINGAPARTMENTS_MODE          213514               69.4
...                                  ...                ...
EXT_SOURCE_2                         660                0.2
AMT_GOODS_PRICE                      278                0.1
AMT_ANNUITY                           12                0.0
CNT_FAM_MEMBERS                        2                0.0
DAYS_LAST_PHONE_CHANGE                 1                0.0

[67 rows x 2 columns]
We have deleted according to your threshold 41 columns,
 here's the new Dataframe


(        SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
 0           100002       1         Cash loans           M            N   
 1           100003       0         Cash loans           F            N   
 2           100004       0    Revolving loans           M            Y   
 3           100006       0         Cash loans           F            N   
 4           100007       0         Cash loans           M            N   
 ...            ...     ...                ...         ...          ...   
 307506      456251       0         Cash loans           M            N   
 307507      456252       0         Cash loans           F            N   
 307508      456253       0         Cash loans           F            N   
 307509      456254       1         Cash loans           F            N   
 307510      456255       0         Cash loans           F            N   
 
        FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  \
 0                    Y    